# **Import packages**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Activation, Input, Add, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate, Lambda, add, GlobalAveragePooling2D, Convolution2D, LocallyConnected2D, ZeroPadding2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
import time
import os

import math
import copy

import cv2
import dlib

# **Load Data**

## Clone Repository

In [2]:
# It's necessary to clone this repository because it  contains the images
!git clone https://github.com/pauldamsa/Face-Recognition.git

Cloning into 'Face-Recognition'...
remote: Enumerating objects: 9575, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 9575 (delta 19), reused 9 (delta 2), pack-reused 9531
Receiving objects: 100% (9575/9575), 177.91 MiB | 23.62 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Checking out files: 100% (6773/6773), done.


## Load Images

In [ ]:
class Image:
  """
    This class will be used for mantaining informations about an image
  """
  def __init__(self, person_name):

    # Name of person
    self.person_name = person_name

    # A list with images of the person
    self.person_images = []

    # list of embeddings
    self.person_embeddings = None

    # a list of dicts with distances
    self.person_distances = None

    # reference images- a list 
    self.reference_embeddings = None

    # images with the faces of the person
    self.person_faces = None

    # number of ref images
    self.number_of_ref_images = 0
  
  def getPersonName(self):
    return self.person_name

  def getPersonImages(self):
    return self.person_images
  
  def setEmbeddings(self, embeddings):
    self.person_embeddings = embeddings
  
  def setDistances(self, distances):
    self.person_distances = distances

  def getReferenceEmbeddings(self):
    return self.reference_embeddings

  def setReferenceImages(self, number_of_ref_images):
    """
      This function is used of setting the ref embeddings for a person
    """
    self.number_of_ref_images = number_of_ref_images
    self.reference_embeddings = self.person_embeddings[:number_of_ref_images]
  
  def setPersonFaces(self, faces):
    self.person_faces = faces
  
  def getPersonFaces(self):
    return self.person_faces
  
  def getEmbeddings(self):
    return self.person_embeddings 
  
  def getDistances(self):
    return self.person_distances
  
  def getReferenceImages(self):
    return self.person_images[:self.number_of_ref_images]
  
  def addImage(self, image):
    """
      This function is used for adding an image with person to his/her list

      Arguments:
        image: the image we want to add (numpy array)
    """
    self.person_images.append(image)

def testImageClass():
  """
    This function is used for testing the Image class
  """
  img = Image("Paul")

  assert img.getPersonName() == "Paul"
  assert img.getPersonName() != "awd"
  assert len(img.getPersonImages()) == 0

  img.addImage(3)
  assert len(img.getPersonImages()) == 1

testImageClass()

class RepositoryImages:
  """
    This class will be used for storing the images
  """
  def __init__(self):

    # A dictionary in this form {name_of_person: Image(name_of_person, person_images) }
    self.images = {}

  def getAllImages(self):
    """
      This function is used for getting all images 

      Return:
        A list with all images from dict that contains Image objects
    """
    return list(self.images.values())
  
  def getAllImagesPersons(self):
    """
      This function is used for getting all images as a list

      Return:
        A list with all images of persons
    """
    imgs = []
    # Iterate through personalities
    for key_image in self.images.keys():
      imgs.append(self.images[key_image].getPersonImages())
    
    # Flat the imgs list because it is a list of lists
    flat_list = []
    for sublist in imgs:
        for item in sublist:
            flat_list.append(item)
    
    return flat_list
  
  def find(self, name_person):
    """
      This function is used for getting the images of a person

      Arguments:
        name_person: a string which represents the name of the person that we want to find
      
      Return:
        A object Image
    """
    return self.images[name_person]
  
  def read(self, path_to_dataset):
    """
      This function is used for reading the images from directories

      Arguments:
        path_to_dataset: the path where the images are stored
    """
    directories = os.listdir(path_to_dataset)

    self.images = {}
    print("Total directories: " +str(len(directories)))

    # Iterate thorugh directories of path_to_dataset directory
    for directory in directories:

      # Get the name of each directory
      name_of_images = os.listdir(path_to_dataset + directory)
      print("Current directory: " + directory + " " + str(len(name_of_images)))

      # Iterate through images
      for name_of_image in name_of_images:

        # If the image is already in dictionary
        if directory in self.images.keys():

          # Read the image
          image = cv2.imread(path_to_dataset + directory + "/" + name_of_image)

          # Add new image 
          self.images[directory].addImage(image)

        else:
          # Read the image
          image = Image(directory)
          image.addImage(cv2.imread(path_to_dataset + directory + "/" + name_of_image))
          self.images[directory] = image

def testRepository():
  # assert len(repo.getAllImages()) == 2756
  assert len(repo.getAllImages()) == 5749

In [4]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Create repository
repo = RepositoryImages()

path_to_dataset = "gdrive/My Drive/DS440-Team3/Code/FaceNet/lfw/raw/"
repo.read(path_to_dataset)

testRepository()

Streaming output truncated to the last 5000 lines.
Current directory: Matt_Anderson 1
Current directory: Jose_Luis_Santiago_Vasconcelos 1
Current directory: Munir_Akram 2
Current directory: Leon_Barmore 1
Current directory: Alex_Zanardi 1
Current directory: Vladimir_Spidla 3
Current directory: Monica_Lewinsky 3
Current directory: Jennifer_Lopez 21
Current directory: Jane_Pauley 2
Current directory: Byron_Scott 2
Current directory: Graciano_Rocchigiani 1
Current directory: Thomas_Ulrich 1
Current directory: Jong_Wook_Lee 4
Current directory: Tom_Daschle 25
Current directory: Chen_Shui-bian 5
Current directory: Mary_Matalin 1
Current directory: David_Ho 1
Current directory: James_Morris 2
Current directory: Valentino_Rossi 6
Current directory: Wu_Yi 3
Current directory: Dan_Wheldon 2
Current directory: Jim_OBrien 3
Current directory: Patty_Schnyder 4
Current directory: John_Moe 1
Current directory: Carol_Niedermayer 1
Current directory: Fidel_Castro 18
Current directory: Tora_Takagi 1
Cu

# **Display images**

## Declarations

In [ ]:
class Service:
  """
    This class will be used for working with the while database of images
  """
  def __init__(self, repo):
    self.repo = repo
  
  def getAllImages(self):
    """
      This function is used for getting all images

      Return:
        A list with all images
    """
    return repo.getAllImages()

  def find(self, name_person):
    """
      This function is used for getting the images of a person

      Arguments:
        name_person: a string which represents the name of the person that we want to find
      
      Return:
        A object Image
    """
    return self.repo.find(name_person)
  
  def getAllImagesPersons(self):
    """
      This function is used for getting all images as a list

      Return:
        A list with all images of persons
    """
    return repo.getAllImagesPersons()
  
  def detectFacesOfPersons(self, face_detector):
    """
      This function will be used in order to get the faces of persons

      Arguments:
        face_detector: dlib face detector
    """

    # Get images
    images = self.getAllImages()

    # Iterate through personalities (Image Objects)
    for current_image in images:
      
      # Get images of current personality
      images_of_current_person = current_image.getPersonImages()

      # Init list of faces
      list_of_faces = []

      # Iterate through images of current personality
      for current_image_person in images_of_current_person:

        # Make the image gray
        image_gray = cv2.cvtColor(current_image_person, cv2.COLOR_RGB2GRAY)

        # Detect faces
        face_coord = face_detector(image_gray, 1)

        # If there was a face detected
        if len(face_coord) > 0:
          (x, y, w, h) = self.rect_to_bb(face_coord[0])
          x1, x2, y1, y2 = self.apply_offsets((x, y, w, h), (10, 10))

          # Crop the face
          image_face = current_image_person[y1:y2, x1:x2, :]

          image_face = np.asarray(image_face, dtype="float32")
          if image_face.shape[0] != 0 and image_face.shape[1] and image_face.shape[2] != 0:

            # Resize the image in (160, 160, 3)
            image_resized = cv2.resize(image_face, (160, 160), interpolation = cv2.INTER_AREA)

            # Normalize the pixels
            image_resized /= 255 

            # append current face to the list of faces
            list_of_faces.append(image_resized)
      
      # Set faces to the current image
      current_image.setPersonFaces(list_of_faces)
  
  def calculateEmbeddings(self, model):
    """
      This function is used for gettinf the embeddings of faces for each person

      Arguments:
        model: the model that will be used for predicting the embeddings
    """
    # Get all images objects
    images = self.getAllImages()

    # Iterate through objects
    for image in images:

      # Get current face images of personality
      current_faces_person = np.asarray(image.getPersonFaces(), dtype="float32")
      print(image.getPersonName() + " " + str(current_faces_person.shape))
      if current_faces_person.shape[0] != 0 and current_faces_person.shape[1] != 0 and current_faces_person.shape[2] != 0:
        # Get embeddings of current face
        current_embeddings = model.predict(current_faces_person, verbose = 0) # verbose = 0 means that we don't want to show the output
        
        # Set embeddings to the current image
        image.setEmbeddings(current_embeddings)
  
  def calculateDistances(self, name_of_person, number_of_ref_images):
    """
      This function is used in order to calculate the distance between a set of reference images and others

      Arguments:
        name_of_person: the name of the person that we want to get the distances
        number_of_ref_images: how many images we want to use as reference
    """
    # Init the list of dictionaries
    list_dist =[]
    # Get image of the person
    image_person = self.find(name_of_person)
    images = self.getAllImages()
    # Set reference images for person
    image_person.setReferenceImages(number_of_ref_images)

    # Get reference embeddings
    ref_embeddings = image_person.getReferenceEmbeddings()
    index = 0
    # Iterate through reference embeddings
    for current_ref_embeddings in ref_embeddings:
      print(index)
      # Init dictionary of distances
      dict_dist = {}

      # Iterate through all images
      for current_image in images:
        # Init list of distances
        list_distances = []

        # Get the current list of embeddings
        list_current_embeddings = current_image.getEmbeddings()
        # Iterate through embeddings of current image
        if len(current_image.getPersonFaces()) > 0:
          for current_image_embedding in list_current_embeddings:
            # Calculate the distance between current reference embeddings and current embeddings
            distance = self.findCosineDistance(current_ref_embeddings, current_image_embedding)

            # append to list of distances
            list_distances.append(distance)
        
          # Insert to dictionary the distances
          dict_dist[current_image.getPersonName()] = list_distances
    
      # Append to list the dictionary
      list_dist.append(dict_dist)

      # Update Index
      index += 1

    # Set the list of dist
    image_person.setDistances(list_dist)

  def make_flat(self,input_list):
    """
      input_list: a list of lists
    """
    flat_list = []
    for sublist in input_list:
        for item in sublist:
            flat_list.append(item)
    return flat_list
  
  def get_data_frame_of_person(self, name_person, threshold = 0.40):
    """
      This function will be used in order to get the dataframe with the performances of the model for a person

      Arguments:
        name_of_person: the name of the person that we want to get the distances
        threshold: the threshold we want to verify
      
      Return:
        A dataframe
    """
    # Init rows of dataframe
    rows = []
    
    # Get distances
    person = self.find(name_person)
    distances_of_person = person.getDistances()

    index = 0
    # Iterate through distances
    for current_dict_distances in distances_of_person:
      # get the list with all distances
      flat_list_distances = self.make_flat(current_dict_distances.values())
      current_list_dist_person = current_dict_distances[name_person]

      number_of_images_person = len(current_list_dist_person)
      total_faces = len(flat_list_distances) - number_of_images_person

      tp = len([x for x in current_list_dist_person if x <= threshold])
      percent_tp = round((tp * 100) / number_of_images_person,2)

      fn = number_of_images_person - tp
      percent_fn  = round(100 - percent_tp,2)
      
      fp = abs(len([x for x in flat_list_distances if x <= threshold]) - tp)
      percent_fp = round((fp * 100 )/ total_faces,2)

      row = [name_person + "_" +str(index), number_of_images_person, tp, percent_tp, fn, percent_fn, fp, total_faces, percent_fp]
      rows.append(row)
      index += 1

    # Create the dataframe
    df = pd.DataFrame(data = rows, columns=["Name", "Images of Name", "True Positives","Percent TP", "False Negatives", "Percent FN", "False positives","Total faces", "Percent FP"])
    
    return df

  def apply_offsets(self, face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return x - x_off, x + width + x_off, y - y_off, y + height + y_off

  def rect_to_bb(self, rect):
      x = rect.left()
      y = rect.top()
      w = rect.right() - x
      h = rect.bottom() - y

      return x, y, w, h

  def findCosineDistance(self, source_representation, test_representation):
    a = np.matmul(np.transpose(copy.deepcopy(source_representation)), copy.deepcopy(test_representation))
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
  
  def displayFacesOfPerson(self, person_name):
    """
      This function is used for displaying the faces of a person

      Arguments:
        person_name: the name of the person that we want to see the faces
    """
    # Define the dimensions of the plot grid 
    list_of_images = self.find(person_name).getPersonFaces()
    dim = math.floor(math.sqrt(len(list_of_images)))
    W_grid = dim
    L_grid = dim

    # fig, axes = plt.subplots(L_grid, W_grid)
    # subplot return the figure object and axes object
    # we can use the axes object to plot specific figures at various locations

    fig, axes = plt.subplots(L_grid, W_grid, figsize = (25,25))

    axes = axes.ravel() 

    index = 0
    for i in np.arange(0, W_grid * L_grid):

        # read and display an image with the selected index    
        axes[i].imshow(np.int8(list_of_images[index]))
        axes[i].set_title(person_name + "_" + str(i))
        index+=1
        axes[i].axis('off')

    plt.subplots_adjust(hspace=0.4)
  
  def displayImages(self, number_of_images):
    """
      This function is used for displaying a number of images from dataset in a grid format

      Arguments:
        number_of_images: int which represents how many images
    """
    images = self.getAllImagesPersons()[:number_of_images]

    # Define the dimensions of the plot grid 
    W_grid = 10
    L_grid = 10

    # fig, axes = plt.subplots(L_grid, W_grid)
    # subplot return the figure object and axes object
    # we can use the axes object to plot specific figures at various locations

    fig, axes = plt.subplots(L_grid, W_grid, figsize = (25,25))

    axes = axes.ravel() 

    index = 0
    for i in np.arange(0, W_grid * L_grid):

        # read and display an image with the selected index    
        axes[i].imshow(cv2.cvtColor(np.uint8(images[index]), cv2.COLOR_BGR2RGB))
        index+=1
        axes[i].axis('off')

    plt.subplots_adjust(hspace=0.4)
  
  def displayImagesOfPerson(self, name_person):
    """
      This function is used for displaying the images of a person

      Arguments:
        name_person: a string which represents the name of a person that we want to see his/her images
    """
    # Define the dimensions of the plot grid 
    list_of_images = self.find(name_person).getPersonImages()
    dim = math.floor(math.sqrt(len(list_of_images)))
    W_grid = dim
    L_grid = dim

    # fig, axes = plt.subplots(L_grid, W_grid)
    # subplot return the figure object and axes object
    # we can use the axes object to plot specific figures at various locations

    fig, axes = plt.subplots(L_grid, W_grid, figsize = (25,25))

    axes = axes.ravel() 

    index = 0
    for i in np.arange(0, W_grid * L_grid):

        # read and display an image with the selected index    
        axes[i].imshow(cv2.cvtColor(np.uint8(list_of_images[index]), cv2.COLOR_BGR2RGB))
        axes[i].set_title(name_person + "_" + str(i))
        index+=1
        axes[i].axis('off')

    plt.subplots_adjust(hspace=0.4)
  
  def displayReferenceImagesOfPerson(self, name_person):
    """
      This function is used for displaying the reference images of a person

      Arguments:
        name_person: a string which represents the name of a person that we want to see his/her images
    """
    # Define the dimensions of the plot grid 
    list_of_images = self.find(name_person).getReferenceImages()
    dim = math.floor(math.sqrt(len(list_of_images)))
    W_grid = dim
    L_grid = dim

    # fig, axes = plt.subplots(L_grid, W_grid)
    # subplot return the figure object and axes object
    # we can use the axes object to plot specific figures at various locations

    fig, axes = plt.subplots(L_grid, W_grid, figsize = (25,25))

    axes = axes.ravel() 

    index = 0
    for i in np.arange(0, W_grid * L_grid):

        # read and display an image with the selected index    
        axes[i].imshow(cv2.cvtColor(np.uint8(list_of_images[index]), cv2.COLOR_BGR2RGB))
        axes[i].set_title(name_person + "_" + str(i))
        index+=1
        axes[i].axis('off')

    plt.subplots_adjust(hspace=0.4)

In [ ]:
service = Service(repo=repo)

## Display first 100 images

In [ ]:
service.displayImages(100)

Output hidden; open in https://colab.research.google.com to view.

## Display images with Tony Blair



In [ ]:
service.displayImagesOfPerson("Tony_Blair")

Output hidden; open in https://colab.research.google.com to view.

# **Get Faces**

## Detect faces of persons

In [ ]:
# This will take around 7min to finish
face_detector = dlib.get_frontal_face_detector()
service.detectFacesOfPersons(face_detector)

# **FaceNet**

In [ ]:
def scaling(x, scale):
	return x * scale

def InceptionResNetV2():
	
	inputs = Input(shape=(160, 160, 3))
	x = Conv2D(32, 3, strides=2, padding='valid', use_bias=False, name= 'Conv2d_1a_3x3') (inputs)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_1a_3x3_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_1a_3x3_Activation')(x)
	x = Conv2D(32, 3, strides=1, padding='valid', use_bias=False, name= 'Conv2d_2a_3x3') (x)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_2a_3x3_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_2a_3x3_Activation')(x)
	x = Conv2D(64, 3, strides=1, padding='same', use_bias=False, name= 'Conv2d_2b_3x3') (x)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_2b_3x3_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_2b_3x3_Activation')(x)
	x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
	x = Conv2D(80, 1, strides=1, padding='valid', use_bias=False, name= 'Conv2d_3b_1x1') (x)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_3b_1x1_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_3b_1x1_Activation')(x)
	x = Conv2D(192, 3, strides=1, padding='valid', use_bias=False, name= 'Conv2d_4a_3x3') (x)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_4a_3x3_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_4a_3x3_Activation')(x)
	x = Conv2D(256, 3, strides=2, padding='valid', use_bias=False, name= 'Conv2d_4b_3x3') (x)
	x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Conv2d_4b_3x3_BatchNorm')(x)
	x = Activation('relu', name='Conv2d_4b_3x3_Activation')(x)
	
	# 5x Block35 (Inception-ResNet-A block):
	branch_0 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block35_1_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_1_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_1_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_2 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_1_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_1_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_1_Branch_2_Conv2d_0c_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_1_Branch_2_Conv2d_0c_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_1_Branch_2_Conv2d_0c_3x3_Activation')(branch_2)
	branches = [branch_0, branch_1, branch_2]
	mixed = Concatenate(axis=3, name='Block35_1_Concatenate')(branches)
	up = Conv2D(256, 1, strides=1, padding='same', use_bias=True, name= 'Block35_1_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.17})(up)
	x = add([x, up])
	x = Activation('relu', name='Block35_1_Activation')(x)
	
	branch_0 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block35_2_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_2_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_2_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_2 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_2_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_2_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_2_Branch_2_Conv2d_0c_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_2_Branch_2_Conv2d_0c_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_2_Branch_2_Conv2d_0c_3x3_Activation')(branch_2)
	branches = [branch_0, branch_1, branch_2]
	mixed = Concatenate(axis=3, name='Block35_2_Concatenate')(branches)
	up = Conv2D(256, 1, strides=1, padding='same', use_bias=True, name= 'Block35_2_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.17})(up)
	x = add([x, up])
	x = Activation('relu', name='Block35_2_Activation')(x)
	
	branch_0 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block35_3_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_3_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_3_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_2 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_3_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_3_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_3_Branch_2_Conv2d_0c_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_3_Branch_2_Conv2d_0c_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_3_Branch_2_Conv2d_0c_3x3_Activation')(branch_2)
	branches = [branch_0, branch_1, branch_2]
	mixed = Concatenate(axis=3, name='Block35_3_Concatenate')(branches)
	up = Conv2D(256, 1, strides=1, padding='same', use_bias=True, name= 'Block35_3_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.17})(up)
	x = add([x, up])
	x = Activation('relu', name='Block35_3_Activation')(x)
	
	branch_0 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block35_4_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_4_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_4_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_2 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_4_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_4_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_4_Branch_2_Conv2d_0c_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_4_Branch_2_Conv2d_0c_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_4_Branch_2_Conv2d_0c_3x3_Activation')(branch_2)
	branches = [branch_0, branch_1, branch_2]
	mixed = Concatenate(axis=3, name='Block35_4_Concatenate')(branches)
	up = Conv2D(256, 1, strides=1, padding='same', use_bias=True, name= 'Block35_4_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.17})(up)
	x = add([x, up])
	x = Activation('relu', name='Block35_4_Activation')(x)
	
	branch_0 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block35_5_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_5_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block35_5_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_2 = Conv2D(32, 1, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_5_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_5_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(32, 3, strides=1, padding='same', use_bias=False, name= 'Block35_5_Branch_2_Conv2d_0c_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block35_5_Branch_2_Conv2d_0c_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Block35_5_Branch_2_Conv2d_0c_3x3_Activation')(branch_2)
	branches = [branch_0, branch_1, branch_2]
	mixed = Concatenate(axis=3, name='Block35_5_Concatenate')(branches)
	up = Conv2D(256, 1, strides=1, padding='same', use_bias=True, name= 'Block35_5_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.17})(up)
	x = add([x, up])
	x = Activation('relu', name='Block35_5_Activation')(x)

	# Mixed 6a (Reduction-A block):
	branch_0 = Conv2D(384, 3, strides=2, padding='valid', use_bias=False, name= 'Mixed_6a_Branch_0_Conv2d_1a_3x3') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_6a_Branch_0_Conv2d_1a_3x3_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Mixed_6a_Branch_0_Conv2d_1a_3x3_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Mixed_6a_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_6a_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Mixed_6a_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, 3, strides=1, padding='same', use_bias=False, name= 'Mixed_6a_Branch_1_Conv2d_0b_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_6a_Branch_1_Conv2d_0b_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Mixed_6a_Branch_1_Conv2d_0b_3x3_Activation')(branch_1)
	branch_1 = Conv2D(256, 3, strides=2, padding='valid', use_bias=False, name= 'Mixed_6a_Branch_1_Conv2d_1a_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_6a_Branch_1_Conv2d_1a_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Mixed_6a_Branch_1_Conv2d_1a_3x3_Activation')(branch_1)
	branch_pool = MaxPooling2D(3, strides=2, padding='valid', name='Mixed_6a_Branch_2_MaxPool_1a_3x3')(x)
	branches = [branch_0, branch_1, branch_pool]
	x = Concatenate(axis=3, name='Mixed_6a')(branches)

	# 10x Block17 (Inception-ResNet-B block):
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_1_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_1_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_1_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_1_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_1_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_1_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_1_Branch_1_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_1_Branch_1_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_1_Branch_1_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_1_Branch_1_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_1_Branch_1_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_1_Branch_1_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_1_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_1_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_1_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_2_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_2_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_2_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_2_Branch_2_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_2_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_2_Branch_2_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_2_Branch_2_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_2_Branch_2_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_2_Branch_2_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_2_Branch_2_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_2_Branch_2_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_2_Branch_2_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_2_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_2_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_2_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_3_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_3_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_3_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_3_Branch_3_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_3_Branch_3_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_3_Branch_3_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_3_Branch_3_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_3_Branch_3_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_3_Branch_3_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_3_Branch_3_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_3_Branch_3_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_3_Branch_3_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_3_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_3_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_3_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_4_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_4_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_4_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_4_Branch_4_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_4_Branch_4_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_4_Branch_4_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_4_Branch_4_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_4_Branch_4_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_4_Branch_4_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_4_Branch_4_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_4_Branch_4_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_4_Branch_4_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_4_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_4_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_4_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_5_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_5_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_5_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_5_Branch_5_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_5_Branch_5_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_5_Branch_5_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_5_Branch_5_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_5_Branch_5_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_5_Branch_5_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_5_Branch_5_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_5_Branch_5_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_5_Branch_5_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_5_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_5_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_5_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_6_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_6_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_6_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_6_Branch_6_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_6_Branch_6_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_6_Branch_6_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_6_Branch_6_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_6_Branch_6_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_6_Branch_6_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_6_Branch_6_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_6_Branch_6_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_6_Branch_6_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_6_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_6_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_6_Activation')(x)	
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_7_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_7_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_7_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_7_Branch_7_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_7_Branch_7_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_7_Branch_7_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_7_Branch_7_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_7_Branch_7_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_7_Branch_7_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_7_Branch_7_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_7_Branch_7_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_7_Branch_7_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_7_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_7_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_7_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_8_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_8_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_8_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_8_Branch_8_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_8_Branch_8_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_8_Branch_8_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_8_Branch_8_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_8_Branch_8_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_8_Branch_8_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_8_Branch_8_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_8_Branch_8_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_8_Branch_8_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_8_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_8_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_8_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_9_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_9_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_9_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_9_Branch_9_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_9_Branch_9_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_9_Branch_9_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_9_Branch_9_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_9_Branch_9_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_9_Branch_9_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_9_Branch_9_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_9_Branch_9_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_9_Branch_9_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_9_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_9_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_9_Activation')(x)
	
	branch_0 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_10_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_10_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block17_10_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(128, 1, strides=1, padding='same', use_bias=False, name= 'Block17_10_Branch_10_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_10_Branch_10_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_10_Branch_10_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(128, [1, 7], strides=1, padding='same', use_bias=False, name= 'Block17_10_Branch_10_Conv2d_0b_1x7') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_10_Branch_10_Conv2d_0b_1x7_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_10_Branch_10_Conv2d_0b_1x7_Activation')(branch_1)
	branch_1 = Conv2D(128, [7, 1], strides=1, padding='same', use_bias=False, name= 'Block17_10_Branch_10_Conv2d_0c_7x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block17_10_Branch_10_Conv2d_0c_7x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block17_10_Branch_10_Conv2d_0c_7x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block17_10_Concatenate')(branches)
	up = Conv2D(896, 1, strides=1, padding='same', use_bias=True, name= 'Block17_10_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.1})(up)
	x = add([x, up])
	x = Activation('relu', name='Block17_10_Activation')(x)

	# Mixed 7a (Reduction-B block): 8 x 8 x 2080	
	branch_0 = Conv2D(256, 1, strides=1, padding='same', use_bias=False, name= 'Mixed_7a_Branch_0_Conv2d_0a_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_0_Conv2d_0a_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Mixed_7a_Branch_0_Conv2d_0a_1x1_Activation')(branch_0)
	branch_0 = Conv2D(384, 3, strides=2, padding='valid', use_bias=False, name= 'Mixed_7a_Branch_0_Conv2d_1a_3x3') (branch_0)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_0_Conv2d_1a_3x3_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Mixed_7a_Branch_0_Conv2d_1a_3x3_Activation')(branch_0)
	branch_1 = Conv2D(256, 1, strides=1, padding='same', use_bias=False, name= 'Mixed_7a_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Mixed_7a_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(256, 3, strides=2, padding='valid', use_bias=False, name= 'Mixed_7a_Branch_1_Conv2d_1a_3x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_1_Conv2d_1a_3x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Mixed_7a_Branch_1_Conv2d_1a_3x3_Activation')(branch_1)
	branch_2 = Conv2D(256, 1, strides=1, padding='same', use_bias=False, name= 'Mixed_7a_Branch_2_Conv2d_0a_1x1') (x)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Mixed_7a_Branch_2_Conv2d_0a_1x1_Activation')(branch_2)
	branch_2 = Conv2D(256, 3, strides=1, padding='same', use_bias=False, name= 'Mixed_7a_Branch_2_Conv2d_0b_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_2_Conv2d_0b_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Mixed_7a_Branch_2_Conv2d_0b_3x3_Activation')(branch_2)
	branch_2 = Conv2D(256, 3, strides=2, padding='valid', use_bias=False, name= 'Mixed_7a_Branch_2_Conv2d_1a_3x3') (branch_2)
	branch_2 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Mixed_7a_Branch_2_Conv2d_1a_3x3_BatchNorm')(branch_2)
	branch_2 = Activation('relu', name='Mixed_7a_Branch_2_Conv2d_1a_3x3_Activation')(branch_2)
	branch_pool = MaxPooling2D(3, strides=2, padding='valid', name='Mixed_7a_Branch_3_MaxPool_1a_3x3')(x)
	branches = [branch_0, branch_1, branch_2, branch_pool]
	x = Concatenate(axis=3, name='Mixed_7a')(branches)

	# 5x Block8 (Inception-ResNet-C block):
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_1_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_1_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_1_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_1_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_1_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_1_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_1_Branch_1_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_1_Branch_1_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_1_Branch_1_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_1_Branch_1_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_1_Branch_1_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_1_Branch_1_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_1_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_1_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.2})(up)
	x = add([x, up])
	x = Activation('relu', name='Block8_1_Activation')(x)
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_2_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_2_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_2_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_2_Branch_2_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_2_Branch_2_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_2_Branch_2_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_2_Branch_2_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_2_Branch_2_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_2_Branch_2_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_2_Branch_2_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_2_Branch_2_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_2_Branch_2_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_2_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_2_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.2})(up)
	x = add([x, up])
	x = Activation('relu', name='Block8_2_Activation')(x)
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_3_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_3_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_3_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_3_Branch_3_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_3_Branch_3_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_3_Branch_3_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_3_Branch_3_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_3_Branch_3_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_3_Branch_3_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_3_Branch_3_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_3_Branch_3_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_3_Branch_3_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_3_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_3_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.2})(up)
	x = add([x, up])
	x = Activation('relu', name='Block8_3_Activation')(x)
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_4_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_4_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_4_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_4_Branch_4_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_4_Branch_4_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_4_Branch_4_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_4_Branch_4_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_4_Branch_4_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_4_Branch_4_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_4_Branch_4_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_4_Branch_4_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_4_Branch_4_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_4_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_4_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.2})(up)
	x = add([x, up])
	x = Activation('relu', name='Block8_4_Activation')(x)
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_5_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_5_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_5_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_5_Branch_5_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_5_Branch_5_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_5_Branch_5_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_5_Branch_5_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_5_Branch_5_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_5_Branch_5_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_5_Branch_5_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_5_Branch_5_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_5_Branch_5_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_5_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_5_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 0.2})(up)
	x = add([x, up])
	x = Activation('relu', name='Block8_5_Activation')(x)
	
	branch_0 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_6_Branch_0_Conv2d_1x1') (x)
	branch_0 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_6_Branch_0_Conv2d_1x1_BatchNorm')(branch_0)
	branch_0 = Activation('relu', name='Block8_6_Branch_0_Conv2d_1x1_Activation')(branch_0)
	branch_1 = Conv2D(192, 1, strides=1, padding='same', use_bias=False, name= 'Block8_6_Branch_1_Conv2d_0a_1x1') (x)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_6_Branch_1_Conv2d_0a_1x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_6_Branch_1_Conv2d_0a_1x1_Activation')(branch_1)
	branch_1 = Conv2D(192, [1, 3], strides=1, padding='same', use_bias=False, name= 'Block8_6_Branch_1_Conv2d_0b_1x3') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_6_Branch_1_Conv2d_0b_1x3_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_6_Branch_1_Conv2d_0b_1x3_Activation')(branch_1)
	branch_1 = Conv2D(192, [3, 1], strides=1, padding='same', use_bias=False, name= 'Block8_6_Branch_1_Conv2d_0c_3x1') (branch_1)
	branch_1 = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001, scale=False, name='Block8_6_Branch_1_Conv2d_0c_3x1_BatchNorm')(branch_1)
	branch_1 = Activation('relu', name='Block8_6_Branch_1_Conv2d_0c_3x1_Activation')(branch_1)
	branches = [branch_0, branch_1]
	mixed = Concatenate(axis=3, name='Block8_6_Concatenate')(branches)
	up = Conv2D(1792, 1, strides=1, padding='same', use_bias=True, name= 'Block8_6_Conv2d_1x1') (mixed)
	up = Lambda(scaling, output_shape=K.int_shape(up)[1:], arguments={'scale': 1})(up)
	x = add([x, up])
	
	# Classification block
	x = GlobalAveragePooling2D(name='AvgPool')(x)
	x = Dropout(1.0 - 0.8, name='Dropout')(x)
	# Bottleneck
	x = Dense(128, use_bias=False, name='Bottleneck')(x)
	x = BatchNormalization(momentum=0.995, epsilon=0.001, scale=False, name='Bottleneck_BatchNorm')(x)

	# Create model
	model = Model(inputs, x, name='inception_resnet_v1')

	return model

# Create the FaceNet model
FaceNetModel = InceptionResNetV2()

# Load the weights of the model
path = "/content/Face-Recognition/facenet_weights.h5"
FaceNetModel.load_weights(path)

# **Analysis of distances**

## Calculate embeddings

In [ ]:
# This will take around 5min. to finish
images_embeddings = service.calculateEmbeddings(FaceNetModel)

Streaming output truncated to the last 5000 lines.
Matt_Anderson (1, 160, 160, 3)
Jose_Luis_Santiago_Vasconcelos (1, 160, 160, 3)
Munir_Akram (2, 160, 160, 3)
Leon_Barmore (1, 160, 160, 3)
Alex_Zanardi (1, 160, 160, 3)
Vladimir_Spidla (3, 160, 160, 3)
Monica_Lewinsky (3, 160, 160, 3)
Jennifer_Lopez (21, 160, 160, 3)
Jane_Pauley (2, 160, 160, 3)
Byron_Scott (2, 160, 160, 3)
Graciano_Rocchigiani (1, 160, 160, 3)
Thomas_Ulrich (1, 160, 160, 3)
Jong_Wook_Lee (4, 160, 160, 3)
Tom_Daschle (25, 160, 160, 3)
Chen_Shui-bian (5, 160, 160, 3)
Mary_Matalin (1, 160, 160, 3)
David_Ho (1, 160, 160, 3)
James_Morris (2, 160, 160, 3)
Valentino_Rossi (6, 160, 160, 3)
Wu_Yi (3, 160, 160, 3)
Dan_Wheldon (2, 160, 160, 3)
Jim_OBrien (3, 160, 160, 3)
Patty_Schnyder (4, 160, 160, 3)
John_Moe (1, 160, 160, 3)
Carol_Niedermayer (1, 160, 160, 3)
Fidel_Castro (18, 160, 160, 3)
Tora_Takagi (1, 160, 160, 3)
Mahathir_Mohamad (14, 160, 160, 3)
Pat_Burns (2, 160, 160, 3)
Chris_Neil (1, 160, 160, 3)
Keanu_Reeves (12, 16

## Calculate distances between ref_imgs and other for a personality

In [ ]:
person_name = "Tony_Blair"
number_of_ref_imgs = 144

service.calculateDistances(person_name, number_of_ref_imgs)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: ignored

**Display DataFrame of a person**

In [ ]:
# threshold is automatically set to 0.4
a <- seq(0, 1, by = 0.05)
df_person = service.get_data_frame_of_person(person_name, threshold = a)

In [ ]:
df_person.sort_values(by="Percent TP",ascending=False, ignore_index=True)

**Display reference images of a person**

In [ ]:
service.displayReferenceImagesOfPerson(person_name)

# **General Report**

In [5]:
df = pd.read_csv("/content/Face-Recognition/performance_facenet_lfw.csv")

In [6]:
# Correct the table
for i in range(len(df)):
  df.at[i, 'Total faces'] = df.iloc[i]['Total faces'] - df.iloc[i]['Images of Name']
  df.at[i, 'Percent FP'] = round((df.iloc[i]['False positives'] * 100) / (df.iloc[i]['Total faces']), 2)

In [7]:
# For this table the reference image for each personlaity was the first one
df.sort_values(by="Images of Name",ascending=False, ignore_index=True).head(500)

,Name,Images of Name,True Positives,Percent TP,False Negatives,Percent FN,False positives,Total faces,Percent FP
0,George_W_Bush,526,509,96.77,17,3.23,12,6162,0.19
1,Tony_Blair,144,113,78.47,31,21.53,7,6544,0.11
2,Gerhard_Schroeder,108,75,69.44,33,30.56,16,6580,0.24
3,Junichiro_Koizumi,60,55,91.67,5,8.33,2,6628,0.03
4,Serena_Williams,52,43,82.69,9,17.31,6,6636,0.09
5,Vladimir_Putin,47,42,89.36,5,10.64,8,6641,0.12
6,Jennifer_Capriati,41,11,26.83,30,73.17,3,6647,0.05
7,Laura_Bush,41,28,68.29,13,31.71,0,6647,0.00
8,Hans_Blix,39,38,97.44,1,2.56,41,6649,0.62
9,Alejandro_Toledo,39,1,2.56,38,97.44,0,6649,0.00


In [8]:
df.describe()

,Images of Name,True Positives,Percent TP,False Negatives,Percent FN,False positives,Total faces,Percent FP
count,2727.000000,2727.000000,2727.000000,2727.000000,2727.000000,2727.000000,2727.000000,2727.000000
mean,2.452512,2.085075,94.500836,0.367437,5.499164,5.685735,6685.547488,0.083198
std,11.285978,10.490087,15.677376,1.831876,15.677376,10.142124,11.285978,0.151816
min,1.000000,1.000000,2.560000,0.000000,0.000000,0.000000,6162.000000,0.000000
25%,1.000000,1.000000,100.000000,0.000000,0.000000,1.000000,6686.000000,0.010000
50%,1.000000,1.000000,100.000000,0.000000,0.000000,2.000000,6687.000000,0.030000
75%,2.000000,1.000000,100.000000,0.000000,0.000000,6.000000,6687.000000,0.090000
max,526.000000,509.000000,100.000000,38.000000,97.440000,163.000000,6687.000000,2.440000


# Fairness/Bias Metrics Experimentation

In [9]:
# start of add'l code

# subset first group of observations
sub = df.sort_values(by="Images of Name",ascending=False, ignore_index=True).head(211)

# subgroup top observations in df by gender & label using new var "gender"
df2 = sub.assign(gender = ['M', 'M', 'M', 'M', 'F', 'M', 'F', 'F', 'M', 'M', 
                          'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 
                           'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 
                           'F', 'F', 'M', 'F', 'F', 'M', 'M', 'F', 'M', 'F', 
                           'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'F', 
                           'F', 'F', 'F', 'M', 'F', 'M', 'M', 'F', 'F', 'M', 
                           'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 
                           'M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 
                           'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 
                           'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 
                           'M', 'F', 'F', 'M', 'M', 'M', 'F', 'M', 'F', 'M',
                           'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 
                           'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 
                           'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 
                           'F', 'M', 'F', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 
                           'M', 'F', 'F', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 
                           'M', 'F', 'F', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 
                           'M', 'F', 'M', 'M', 'F', 'M', 'M', 'F', 'F', 'M', 
                           'M', 'M', 'F', 'F', 'F', 'M', 'M', 'F', 'F', 'M', 
                           'M', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 
                           'M', 'M', 'F', 'F', 'F', 'M', 'F', 'M', 'M', 'F']) # next time: start at 211

# W = white, B = black, A = asian
df3 = sub.assign(race = ['W', 'W', 'W', 'A', 'B', 'W', 'W', 'W', 'W', 'W', 
                          'W', 'W', 'W', 'W', 'W', 'B', 'A', 'W', 'W', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'B', 'W', 'W', 'A', 'W', 'W', 
                         'W', 'B', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'B', 'W', 'W', 'B', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'A', 'W', 'A', 'W', 
                         'W', 'W', 'W', 'A', 'A', 'W', 'W', 'W', 'B', 'W', 
                         'W', 'W', 'W', 'W', 'A', 'W', 'W', 'A', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'W', 'A', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'A', 'W', 'W', 'A', 
                         'A', 'W', 'A', 'W', 'A', 'A', 'W', 'W', 'W', 'W', 
                         'W', 'B', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 
                         'W', 'B', 'B', 'W', 'W', 'W', 'W', 'W','W', 'W', 
                         'A', 'W', 'W', 'W', 'W', 'W', 'W', 'W','W','W', 
                         'W', 'W', 'A', 'W', 'W', 'W', 'W', 'W', 'A', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'A', 'W', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 
                         'W', 'W', 'W', 'B', 'A', 'W', 'W', 'A', 'W', 'W', 
                         'W', 'A', 'A', 'W', 'W', 'W', 'W', 'B', 'W', 'W', 
                         'W', 'W', 'W', 'W', 'B', 'W', 'W', 'A', 'W', 'W']) # start at 211

df2.sort_values(by="Images of Name",ascending=False, ignore_index=True).head(500)

,Name,Images of Name,True Positives,Percent TP,False Negatives,Percent FN,False positives,Total faces,Percent FP,gender
0,George_W_Bush,526,509,96.77,17,3.23,12,6162,0.19,M
1,Tony_Blair,144,113,78.47,31,21.53,7,6544,0.11,M
2,Gerhard_Schroeder,108,75,69.44,33,30.56,16,6580,0.24,M
3,Junichiro_Koizumi,60,55,91.67,5,8.33,2,6628,0.03,M
4,Serena_Williams,52,43,82.69,9,17.31,6,6636,0.09,F
5,Vladimir_Putin,47,42,89.36,5,10.64,8,6641,0.12,M
6,Jennifer_Capriati,41,11,26.83,30,73.17,3,6647,0.05,F
7,Laura_Bush,41,28,68.29,13,31.71,0,6647,0.00,F
8,Hans_Blix,39,38,97.44,1,2.56,41,6649,0.62,M
9,Alejandro_Toledo,39,1,2.56,38,97.44,0,6649,0.00,M


In [10]:
df3.sort_values(by="Images of Name",ascending=False, ignore_index=True).head(500)

,Name,Images of Name,True Positives,Percent TP,False Negatives,Percent FN,False positives,Total faces,Percent FP,race
0,George_W_Bush,526,509,96.77,17,3.23,12,6162,0.19,W
1,Tony_Blair,144,113,78.47,31,21.53,7,6544,0.11,W
2,Gerhard_Schroeder,108,75,69.44,33,30.56,16,6580,0.24,W
3,Junichiro_Koizumi,60,55,91.67,5,8.33,2,6628,0.03,A
4,Serena_Williams,52,43,82.69,9,17.31,6,6636,0.09,B
5,Vladimir_Putin,47,42,89.36,5,10.64,8,6641,0.12,W
6,Jennifer_Capriati,41,11,26.83,30,73.17,3,6647,0.05,W
7,Laura_Bush,41,28,68.29,13,31.71,0,6647,0.00,W
8,Hans_Blix,39,38,97.44,1,2.56,41,6649,0.62,W
9,Alejandro_Toledo,39,1,2.56,38,97.44,0,6649,0.00,W


In [11]:
# subsets of dataframe

females = df2[df2["gender"] == 'F'] # 62 observations
males = df2[df2["gender"] == 'M'] # 149 observations

white = df3[df3["race"] == 'W'] # 173 observations
non_white = df3[df3["race"] != 'W'] # 38 observations

In [16]:
# create subsets for balance
males_sub = males.head(62)
white_sub = white.head(38)

In [17]:
# accuracy (equation: (TP+TN)/ (TP+FN+TN+FP))
TN_F = females['Images of Name']-females['True Positives']
TN_M = males['Images of Name']-males['True Positives']
TN_W = white['Images of Name']-white['True Positives']
TN_NW = non_white['Images of Name']-non_white['True Positives']

accuracyF = (females['True Positives'] + TN_F)/(females['True Positives'] + females['False Negatives'] + TN_F + females['False positives'])
accuracyM = (males['True Positives'] + TN_M)/(males['True Positives'] + males['False Negatives'] + TN_M + males['False positives'])
accuracyW = (white['True Positives'] + TN_W)/(white['True Positives'] + white['False Negatives'] + TN_W + white['False positives'])
accuracyNW = (non_white['True Positives'] + TN_NW)/(non_white['True Positives'] + non_white['False Negatives'] + TN_NW + non_white['False positives'])

TN_M2 = males_sub['Images of Name']-males_sub['True Positives']
accuracyM2 = (males_sub['True Positives'] + TN_M2)/(males_sub['True Positives'] + males_sub['False Negatives'] + TN_M2 + males_sub['False positives'])
TN_W2 = white_sub['Images of Name']-white_sub['True Positives']
accuracyW2 = (white_sub['True Positives'] + TN_W2)/(white_sub['True Positives'] + white_sub['False Negatives'] + TN_W2 + white_sub['False positives'])

In [18]:
# precision: tp / (tp + fp)
precF = females['True Positives'] / (females['True Positives'] + females['False positives'])
precM = males['True Positives'] / (males['True Positives'] + males['False positives'])
precW = white['True Positives'] / (white['True Positives'] + white['False positives'])
precNW = non_white['True Positives'] / (non_white['True Positives'] + non_white['False positives'])

precM2 = males_sub['True Positives'] / (males_sub['True Positives'] + males_sub['False positives'])
precW2 = white_sub['True Positives'] / (white_sub['True Positives'] + white_sub['False positives'])

In [19]:
# recall: tp / (tp + fn)
recallF = females['True Positives'] / (females['True Positives'] + females['False Negatives'])
recallM = males['True Positives'] / (males['True Positives'] + males['False Negatives'])
recallW = white['True Positives'] / (white['True Positives'] + white['False Negatives'])
recallNW = non_white['True Positives'] / (non_white['True Positives'] + non_white['False Negatives'])

recallM2 = males_sub['True Positives'] / (males_sub['True Positives'] + males_sub['False Negatives'])
recallW2 = white_sub['True Positives'] / (white_sub['True Positives'] + white_sub['False Negatives'])

In [20]:
from scipy.stats import sem

bounds = {'accuF': 1.96*sem(accuracyF), 'accuM': 1.96*sem(accuracyM), 'accuM2': 1.96*sem(accuracyM2),
          'precF': 1.96*sem(precF), 'precM': 1.96*sem(precM), 'precM2': 1.96*sem(precM2),
          'recallF': 1.96*sem(recallF), 'recallM': 1.96*sem(recallM), 'recallM2': 1.96*sem(recallM2)}
print(bounds)

{'accuF': 0.04863955948842433, 'accuM': 0.03422717173819309, 'accuM2': 0.04405493526751183, 'precF': 0.0664071185991526, 'precM': 0.04414877058974384, 'precM2': 0.06181186595565918, 'recallF': 0.06212650063114369, 'recallM': 0.04091161663021156, 'recallM2': 0.06554649374229092}


In [31]:
bounds2 = {'accuNW': 1.96*sem(accuracyNW), 'accuW': 1.96*sem(accuracyW), 'accuW2': 1.96*sem(accuracyW2),
          'precNW': 1.96*sem(precNW), 'precW': 1.96*sem(precW), 'precW2': 1.96*sem(precW2),
          'recallNW': 1.96*sem(recallNW), 'recallW': 1.96*sem(recallW), 'recallW2': 1.96*sem(recallW2)}
print(bounds2)

{'accuNW': 0.0700722816345047, 'accuW': 0.030596316853952667, 'accuW2': 0.05108396727432422, 'precNW': 0.09155994073140022, 'precW': 0.040554791164910876, 'precW2': 0.07167328074652608, 'recallNW': 0.0765700510428417, 'recallW': 0.0377477415655261, 'recallW2': 0.08563020316938608}


In [21]:
# new df for stats

import pandas as pd

d = {'accuracyF': accuracyF, 'precF': precF, 'recallF': recallF}

statsF = pd.DataFrame(data=d)

print(statsF)

     accuracyF     precF   recallF
4     0.776119  0.877551  0.826923
6     0.554054  0.785714  0.268293
7     0.759259  1.000000  0.682927
16    0.864865  0.885714  0.968750
23    0.750000  0.944444  0.708333
29    0.880000  1.000000  0.863636
31    0.750000  0.818182  0.857143
32    0.724138  1.000000  0.619048
34    0.689655  1.000000  0.550000
35    0.571429  0.777778  0.350000
38    0.703704  1.000000  0.578947
40    0.863636  0.944444  0.894737
47    0.516129  1.000000  0.062500
50    0.516129  0.518519  0.875000
51    0.750000  1.000000  0.666667
52    0.625000  0.714286  0.666667
53    0.882353  0.882353  1.000000
55    0.750000  0.777778  0.933333
58    0.636364  1.000000  0.428571
59    0.233333  0.233333  1.000000
61    0.736842  1.000000  0.642857
63    0.812500  1.000000  0.769231
67    0.279070  0.243243  0.750000
72    0.578947  1.000000  0.272727
75    0.423077  0.416667  0.909091
102   0.444444  0.333333  0.250000
103   0.666667  0.666667  1.000000
107   0.615385  0.66

In [22]:
d2 = {'accuracyM': accuracyM, 'precM': precM, 'recallM': recallM}

statsM = pd.DataFrame(data=d2)

print(statsM)

     accuracyM     precM   recallM
0     0.947748  0.976967  0.967681
1     0.791209  0.941667  0.784722
2     0.687898  0.824176  0.694444
3     0.895522  0.964912  0.916667
5     0.783333  0.840000  0.893617
8     0.481481  0.481013  0.974359
9     0.506494  1.000000  0.025641
10    0.878049  0.969697  0.888889
11    0.837209  0.871795  0.944444
12    0.760870  0.800000  0.914286
13    0.804878  0.931034  0.818182
14    0.804878  0.820513  0.969697
15    0.551724  0.555556  0.937500
17    0.659574  0.666667  0.967742
18    0.756757  1.000000  0.678571
19    0.491228  0.473684  0.321429
20    0.714286  0.758621  0.880000
21    0.545455  0.583333  0.583333
22    0.666667  1.000000  0.500000
24    0.821429  1.000000  0.782609
25    0.638889  1.000000  0.434783
26    0.666667  0.789474  0.681818
27    1.000000  1.000000  1.000000
28    0.687500  1.000000  0.545455
30    0.750000  0.750000  1.000000
33    0.344828  0.218750  0.350000
36    0.513514  0.600000  0.157895
37    0.678571  1.00

In [23]:
d3 = {'accuracyW': accuracyW, 'precW': precW, 'recallW': recallW}
statsW = pd.DataFrame(data=d3)

d4 = {'accuracyNW': accuracyNW, 'precNW': precNW, 'recallNW': recallNW}
statsNM = pd.DataFrame(data=d4)

In [24]:
# overall stats (females)

import statistics as st

print(st.mean(accuracyF))
print(st.mean(precF))
print(st.mean(recallF))

0.6576162121520271
0.7498865422435937
0.7449139818747514


In [25]:
# overall stats (males)
print(st.mean(accuracyM))
print(st.mean(precM))
print(st.mean(recallM))

# overall stats (males, balanced)
print(st.mean(accuracyM2))
print(st.mean(precM2))
print(st.mean(recallM2))

0.6183162035430849
0.6575602509065441
0.7593037221367516
0.6582052290888267
0.7512010404677257
0.7052442497945948


In [26]:
# overall stats (white)
print(st.mean(accuracyW))
print(st.mean(precW))
print(st.mean(recallW))

# overall stats (white, balanced)
print(st.mean(accuracyW2))
print(st.mean(precW2))
print(st.mean(recallW2))

0.6386042120900468
0.6946697480751338
0.7394624683556826
0.6907712405857163
0.8055990615785417
0.673927018025227


In [27]:
# overall stats (non-white)
print(st.mean(accuracyNW))
print(st.mean(precNW))
print(st.mean(recallNW))

0.5900734418359797
0.639252015452098
0.8261556433967762


In [28]:
# disparate impact

# females are the "unprivileged" group
dis_imp = (sum(females['True Positives'])/sum(females['Images of Name'])) / (sum(males['True Positives'])/sum(males['Images of Name']))
print(dis_imp)

# greater than 80% will be an issue
# our result is less than 80% (but close), but may be due to biased data

# note: metric increased a bit when I added 10 more images to test dataset

0.9035284769021532


In [29]:
# disparate impact on race (white vs non_white)

dis_imp2 = (sum(non_white['True Positives'])/sum(non_white['Images of Name'])) / (sum(white['True Positives'])/sum(white['Images of Name']))
print(dis_imp2)

1.0601143041133383


In [30]:
dis_imp3 = (sum(females['True Positives'])/sum(females['Images of Name'])) / (sum(males_sub['True Positives'])/sum(males_sub['Images of Name']))
print(dis_imp3)

dis_imp4 = (sum(non_white['True Positives'])/sum(non_white['Images of Name'])) / (sum(white_sub['True Positives'])/sum(white_sub['Images of Name']))
print(dis_imp4)

0.9060841881853555
1.041476311019907


In [ ]:
# failed trials
# plot ROC curve (need to fix)
import matplotlib.pyplot as plt

# create ROC curve (females)
plt.plot(females['Percent FP'], females['Percent TP'])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# failed trials
# create ROC curve (males)
plt.plot(males['Percent FP'], males['Percent TP'])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()